In [1]:
from google.colab import files
from google.colab import drive
import os
import io
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chisquare
import numpy as np
from datetime import datetime

In [2]:
drive.mount('/content/drive') #/MyDrive/Colab Notebooks/데이터/기상데이터

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
wind = pd.read_csv('drive/MyDrive/db_aws_wind_dd.csv', index_col = 0)
cloud = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/db_cloud.csv', index_col = 0)
flight = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/db_flght.csv', index_col = 0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
rn = pd.read_csv('drive/MyDrive/rn.csv', index_col = 0)
ta = pd.read_csv('drive/MyDrive/ta.csv', index_col = 0)

In [7]:
lon_lat = pd.read_csv('drive/MyDrive/mapping_0713.csv', index_col = 0)

In [8]:
cloud = cloud.rename(columns={'a.tm' : 'tma', 'a.stn_id' : 'stn_id', 'a.clfm_abbr_cd' : 'cloud.clfm_abbr_cd', 'a.ca' : 'cloud.ca', 'a.ch' : 'cloud.ch'})
flight = flight.rename(columns = {'a.tm' : 'tma', 'a.stn_id' : 'stn_id', 'a.max_wd' : 'flight.max_wd', 'a.max_ws' : 'flight.max_ws', 'a.max_ins_wd' : 'flight.max_ins_wd', 'a.max_iws' : 'flight.max_ins_wd', 'a.max_ta' : 'flight.max_ta', 'a.min_ta' : 'flight.min_ta', 'a.max_ps' : 'flight.max_ps', 'a.min_ps' : 'flight.min_ps', 'a.hr1_max_rn' : 'flight.hr1_max_rn'})
rn = rn.rename(columns = {'a.tma' : 'tma', 'a.stn_id' : 'stn_id', 'a.sum_rn' : 'rn.sum_rn', 'a.hr1_max_rn' : 'rn.hr1_max_rn'})
ta = ta.rename(columns = {'a.tma' : 'tma', 'a.stn_id' : 'stn_id', 'a.avg_ta' : 'ta.avg_ta', 'a.max_ta' : 'ta.max_ta', 'a.min_ta' : 'ta.min_ta'})
wind = wind.rename(columns = {'a.tma' : 'tma', 'a.stn_id' : 'stn_id', 'a.avg_ws' : 'wind.avg_ws', 'a.mix_ws' : 'wind.mix_ws', 'a.max_ws' : 'wind.max_ws', 'a.max_wd' : 'wind.max_wd', 'a.max_ws_wd' : 'wind.max_ws_wd', 'a.max_ins_ws' : 'wind.max_ins_ws', 'a.max_ins_ws_wd' : 'wind_max_ins_ws_wd', 'a.mix_wd' : 'wind.mix_wd'})

In [9]:
cloud['date'] = cloud['tma'].map(lambda date_cloud: date_cloud[:10])
cloud['time'] = cloud['tma'].map(lambda time_cloud: time_cloud[11:-2])
cloud['date'] = pd.to_datetime(cloud['date'], format='%Y-%m-%d')
cloud['time'] = pd.to_datetime(cloud['time'], format='%H:%M:%S')

flight['date'] = flight['tma'].map(lambda date_flight: date_flight[:10])
flight['time'] = flight['tma'].map(lambda time_flight: time_flight[11:-2])
flight['date'] = pd.to_datetime(flight['date'], format='%Y-%m-%d')
flight['time'] = pd.to_datetime(flight['time'], format='%H:%M:%S')

rn['date'] = rn['tma'].map(lambda date_rn: date_rn[:10])
rn['time'] = rn['tma'].map(lambda time_rn: time_rn[11:-2])
rn['date'] = pd.to_datetime(rn['date'], format='%Y-%m-%d')
rn['time'] = pd.to_datetime(rn['time'], format='%H:%M:%S')

ta['date'] = ta['tma'].map(lambda date_ta: date_ta[:10])
ta['time'] = ta['tma'].map(lambda time_ta: time_ta[11:-2])
ta['date'] = pd.to_datetime(ta['date'], format='%Y-%m-%d')
ta['time'] = pd.to_datetime(ta['time'], format='%H:%M:%S')

wind['date'] = wind['tma'].map(lambda date_wind: date_wind[:10])
wind['time'] = wind['tma'].map(lambda time_wind: time_wind[11:-2])
wind['date'] = pd.to_datetime(wind['date'], format='%Y-%m-%d')
wind['time'] = pd.to_datetime(wind['time'], format='%H:%M:%S')

In [10]:
# flight테이블에 flight.hr1_max_rn변수는 40292행 중에서 29996행이 결측이라 제거하기로 결정
del flight['flight.hr1_max_rn']

In [11]:
del cloud['tma']
del cloud['time']

del flight['tma']
del flight['time']

del rn['tma']
del rn['time']

del ta['tma']
del ta['time']

del wind['tma']
del wind['time']

In [ ]:
flight_ds = flight.groupby(['date', 'stn_id'], as_index=False).mean()
rn_ds = rn.groupby(['date', 'stn_id'], as_index=False).mean()
ta_ds = ta.groupby(['date', 'stn_id'], as_index=False).mean()
wind_ds = wind.groupby(['date', 'stn_id'], as_index=False).mean()

In [16]:
flight_ds = flight.groupby(['date', 'stn_id'], as_index=False).median()
rn_ds = rn.groupby(['date', 'stn_id'], as_index=False).median()
ta_ds = ta.groupby(['date', 'stn_id'], as_index=False).median()
wind_ds = wind.groupby(['date', 'stn_id'], as_index=False).median()

In [17]:
flight_ds = pd.merge(flight_ds, lon_lat, on = ['stn_id'])
rn_ds = pd.merge(rn_ds, lon_lat, on = ['stn_id'])
ta_ds = pd.merge(ta_ds, lon_lat, on = ['stn_id'])
wind_ds = pd.merge(wind_ds, lon_lat, on = ['stn_id'])

In [ ]:
flight_ds_m = flight_ds.groupby(['date', 'add'], as_index=False).mean()
rn_ds_m = rn_ds.groupby(['date', 'add'], as_index=False).mean()
ta_ds_m = ta_ds.groupby(['date', 'add'], as_index=False).mean()
wind_ds_m = wind_ds.groupby(['date', 'add'], as_index=False).mean()

In [18]:
flight_ds_m = flight_ds.groupby(['date', 'add'], as_index=False).median()
rn_ds_m = rn_ds.groupby(['date', 'add'], as_index=False).median()
ta_ds_m = ta_ds.groupby(['date', 'add'], as_index=False).median()
wind_ds_m = wind_ds.groupby(['date', 'add'], as_index=False).median()

In [ ]:
#ta_ds_m.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/ta.csv')
#rn_ds_m.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/rn.csv')
# flight랑 wind는 시군구별 병합 전 상태에서 조취를 취해야 할 것 같음

# Cloud 전처리

In [19]:
cloud = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/db_cloud.csv', index_col = 0)
lon_lat = pd.read_csv('drive/MyDrive/mapping_0713.csv', index_col = 0)
cloud = cloud.rename(columns={'a.tm' : 'tma', 'a.stn_id' : 'stn_id', 'a.clfm_abbr_cd' : 'cloud.clfm_abbr_cd', 'a.ca' : 'cloud.ca', 'a.ch' : 'cloud.ch'})

cloud['date'] = cloud['tma'].map(lambda date_cloud: date_cloud[:10])
cloud['time'] = cloud['tma'].map(lambda time_cloud: time_cloud[11:-2])
cloud['date'] = pd.to_datetime(cloud['date'], format='%Y-%m-%d')
cloud['time'] = pd.to_datetime(cloud['time'], format='%H:%M:%S')

del cloud['tma']
del cloud['time']

cloud_ds = pd.merge(cloud, lon_lat, on = ['stn_id'])
del cloud_ds['address']

In [20]:
cloud_ds = pd.merge(cloud, lon_lat, on = ['stn_id'])
del cloud_ds['name']
del cloud_ds['address']

In [ ]:
idx = ['cloud.ca', 'cloud.ch']

In [ ]:
area = cloud_ds[['stn_id', 'name', 'lat', 'long', 'add']]

In [ ]:
for col in idx:
    for s in cloud_ds[cloud_ds[col].isnull()]["stn_id"].unique():
        latlon = cloud_ds.query("stn_id == @s")[["lat", "long"]].drop_duplicates().values[0]
        x = (np.cos(latlon[0]) * 6400 * 2 * np.pi / 360) * np.abs(latlon[1] - area[["long"]])
        y = 111 * np.abs(latlon[0] - area["lat"])
        distance = np.sqrt((x**2).squeeze() + y**2).sort_values().head(6)
        dd = pd.concat([area.iloc[distance.index.to_list(), :], pd.DataFrame(distance, columns = ["dist"])], axis = 1)
        #display(dd)
        tg = cloud_ds[cloud_ds["stn_id"] == s]
        ss = dd["stn_id"]
        ii = tg[tg[col].isnull()].index
        ymd = cloud_ds.loc[ii, "date"]
        #display(df.loc[ii, :])
        cloud_ds.loc[ii, col] = np.nanmean(cloud_ds.query("stn_id in @ss and stn_id != @s").pivot(index = "date", columns = "stn_id", values = col).query("date in @ymd"), axis = 1)
        #print(np.nanmean(df.query("stn_id in @ss and stn_id != @s").pivot(index = "yyyymmdd", columns = "stn_id", values = col).query("yyyymmdd in @ymd"), axis = 1))
cloud_ds.isnull().sum()

In [ ]:
cloud_ds

,stn_id,cloud.clfm_abbr_cd,cloud.ca,cloud.ch,date,name,lat,long,add
0,100,Sc,5.0,2.0,2012-02-01,대관령,37.677115,128.71832,강원
1,100,Ci,3.0,55.0,2012-02-03,대관령,37.677115,128.71832,강원
2,100,Sc,7.0,2.0,2012-02-04,대관령,37.677115,128.71832,강원
3,100,Sc,7.0,2.0,2012-02-04,대관령,37.677115,128.71832,강원
4,100,Sc,8.0,2.0,2012-02-04,대관령,37.677115,128.71832,강원
...,...,...,...,...,...,...,...,...,...
1953564,177,Ci,1.0,60.0,2022-10-29,홍성,36.657545,126.68771,충남
1953565,177,Ci,7.0,60.0,2022-10-29,홍성,36.657545,126.68771,충남
1953566,177,St,9.0,3.0,2022-10-30,홍성,36.657545,126.68771,충남
1953567,177,St,5.0,1.0,2022-10-30,홍성,36.657545,126.68771,충남


In [21]:
cloud_ds_m = cloud_ds.groupby(['date', 'add'], as_index=False).median()

In [22]:
set(cloud_ds_m['add']) # (i) 세종이 없음 ** (ii)보간됨!

{'강원',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북'}

In [23]:
cloud_sejong = cloud_ds_m[cloud_ds_m['add'].isin(['충남', '충북', '대전'])].groupby(['date'], as_index=False).mean().assign(add='세종')
cloud_ds_m = cloud_ds_m.append(cloud_sejong)

In [24]:
len(set(cloud_ds_m['date']))

3305

In [ ]:
len(set(rn_ds_m['date']))

3303

In [ ]:
# 날짜는 cloud가 더 많은데 행은 ta랑 rn이 더 길어.. 
# 아마 시군구 중에 날짜가 누락된게 있을 것 같다.

In [25]:
cloud_ds_m.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/cloud_median.csv')

# wind, flight 전처리

In [ ]:
cloud = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/db_cloud.csv', index_col = 0)
lon_lat = pd.read_csv('drive/MyDrive/mapping_0713.csv', index_col = 0)
cloud = cloud.rename(columns={'a.tm' : 'tma', 'a.stn_id' : 'stn_id', 'a.clfm_abbr_cd' : 'cloud.clfm_abbr_cd', 'a.ca' : 'cloud.ca', 'a.ch' : 'cloud.ch'})

cloud['date'] = cloud['tma'].map(lambda date_cloud: date_cloud[:10])
cloud['time'] = cloud['tma'].map(lambda time_cloud: time_cloud[11:-2])
cloud['date'] = pd.to_datetime(cloud['date'], format='%Y-%m-%d')
cloud['time'] = pd.to_datetime(cloud['time'], format='%H:%M:%S')

del cloud['tma']
del cloud['time']

cloud_ds = pd.merge(cloud, lon_lat, on = ['stn_id'])
del cloud_ds['address']

In [ ]:
flight = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/db_flght.csv', index_col = 0)
flight = flight.rename(columns = {'a.tm' : 'tma', 'a.stn_id' : 'stn_id', 'a.max_wd' : 'flight.max_wd', 'a.max_ws' : 'flight.max_ws', 'a.max_ins_wd' : 'flight.max_ins_wd', 'a.max_iws' : 'flight.max_ins_wd', 'a.max_ta' : 'flight.max_ta', 'a.min_ta' : 'flight.min_ta', 'a.max_ps' : 'flight.max_ps', 'a.min_ps' : 'flight.min_ps', 'a.hr1_max_rn' : 'flight.hr1_max_rn'})

flight['date'] = flight['tma'].map(lambda date_flight: date_flight[:10])
flight['time'] = flight['tma'].map(lambda time_flight: time_flight[11:-2])
flight['date'] = pd.to_datetime(flight['date'], format='%Y-%m-%d')
flight['time'] = pd.to_datetime(flight['time'], format='%H:%M:%S')

del flight['tma']
del flight['time']
del flight['flight.hr1_max_rn']

flight_ds = pd.merge(flight, lon_lat, on = ['stn_id'])

In [ ]:
flight_ds.isna().sum()

stn_id               0
flight.max_wd        4
flight.max_ws        4
flight.max_ins_wd    4
flight.max_ins_wd    4
flight.max_ta        4
flight.min_ta        4
flight.max_ps        4
flight.min_ps        4
date                 0
name                 0
address              0
lat                  0
long                 0
add                  0
dtype: int64

In [ ]:
len(flight_ds)

21887

In [ ]:
len(set(flight['date']))

3260

In [ ]:
flight = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/db_flght.csv', index_col = 0)

flight는 누락된 정보가 너무 많아서 폐기

In [26]:
wind = pd.read_csv('drive/MyDrive/db_aws_wind_dd.csv', index_col = 0)
wind = wind.rename(columns = {'a.tma' : 'tma', 'a.stn_id' : 'stn_id', 'a.avg_ws' : 'wind.avg_ws', 'a.mix_ws' : 'wind.mix_ws', 'a.max_ws' : 'wind.max_ws', 'a.max_wd' : 'wind.max_wd', 'a.max_ws_wd' : 'wind.max_ws_wd', 'a.max_ins_ws' : 'wind.max_ins_ws', 'a.max_ins_ws_wd' : 'wind_max_ins_ws_wd', 'a.mix_wd' : 'wind.mix_wd'})

wind['date'] = wind['tma'].map(lambda date_wind: date_wind[:10])
wind['time'] = wind['tma'].map(lambda time_wind: time_wind[11:-2])
wind['date'] = pd.to_datetime(wind['date'], format='%Y-%m-%d')
wind['time'] = pd.to_datetime(wind['time'], format='%H:%M:%S')

del wind['tma']
del wind['time']

wind = pd.merge(wind, lon_lat, on = ['stn_id'])

In [27]:
wind_ds_m = wind_ds.groupby(['date', 'add'], as_index=False).median()

In [32]:
wind_ds_m.isna().sum()

date                  0
add                   0
stn_id                0
wind.avg_ws           0
wind.mix_ws           0
wind.max_ws           0
wind.max_wd           0
wind.max_ws_wd        0
wind.max_ins_ws       0
wind_max_ins_ws_wd    0
wind.mix_wd           0
lat                   0
long                  0
dtype: int64

In [ ]:
print(wind_ds_m[wind_ds_m['wind.max_ws'].isna()])
print(wind_ds_m[wind_ds_m['wind.max_ws_wd'].isna()])
print(wind_ds_m[wind_ds_m['wind.mix_wd'].isna()])

           date add  stn_id  wind.avg_ws  wind.mix_ws  wind.max_ws  \
9270 2013-09-30  대구   665.5          1.6        1.075          NaN   

      wind.max_wd  wind.max_ws_wd  wind.max_ins_ws  wind_max_ins_ws_wd  \
9270      163.125             NaN             5.95              213.75   

      wind.mix_wd        lat        long  
9270          NaN  35.828705  128.559044  
           date add  stn_id  wind.avg_ws  wind.mix_ws  wind.max_ws  \
9270 2013-09-30  대구   665.5          1.6        1.075          NaN   

      wind.max_wd  wind.max_ws_wd  wind.max_ins_ws  wind_max_ins_ws_wd  \
9270      163.125             NaN             5.95              213.75   

      wind.mix_wd        lat        long  
9270          NaN  35.828705  128.559044  
           date add  stn_id  wind.avg_ws  wind.mix_ws  wind.max_ws  \
9270 2013-09-30  대구   665.5          1.6        1.075          NaN   

      wind.max_wd  wind.max_ws_wd  wind.max_ins_ws  wind_max_ins_ws_wd  \
9270      163.125             NaN

In [ ]:
#충남, 충북, 세종, 전북

In [29]:
wind_daejeon = wind_ds_m[wind_ds_m['add'].isin(['충남', '충북', '세종', '전북'])].groupby(['date'], as_index=False).mean().assign(add='세종')
#wind_ds_m = wind_ds_m.append(wind_daejeon)

In [30]:
wind_daejeon[wind_daejeon['date'] == pd.to_datetime('2013-09-30', format='%Y-%m-%d')][['wind.max_ws', 'wind.max_ws_wd', 'wind.mix_wd']]

,wind.max_ws,wind.max_ws_wd,wind.mix_wd
545,2.9625,187.4125,148.9875


In [31]:
wind_ds_m.iloc[9270,5] = 2.9625
wind_ds_m.iloc[9270,7] = 187.4125
wind_ds_m.iloc[9270,10] = 148.9875

In [34]:
wind_ds_m.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/wind_median.csv')

# cloud, rn, ta, wind 데이터 병합하기

In [ ]:
cloud = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/cloud.csv', index_col = 0)
rn = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/rn.csv', index_col = 0)
ta = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/ta.csv', index_col = 0)
wind = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/wind.csv', index_col = 0)

In [ ]:
print(cloud)
print(rn)
print(ta)
print(wind)

In [ ]:
del cloud['stn_id']
del cloud['lat']
del cloud['long']

del rn['stn_id']
del rn['lat']
del rn['long']

del ta['stn_id']
del ta['lat']
del ta['long']

del wind['stn_id']
del wind['lat']
del wind['long']

In [ ]:
cloud_wind = pd.merge(cloud, wind, on = ['date', 'add'])
ta_rn = pd.merge(ta, rn, on = ['date', 'add'])
all_merged = pd.merge(cloud_wind, ta_rn, on = ['date', 'add'])

In [ ]:
all_merged.columns

Index(['date', 'add', 'cloud.ca', 'cloud.ch', 'wind.avg_ws', 'wind.mix_ws',
       'wind.max_ws', 'wind.max_wd', 'wind.max_ws_wd', 'wind.max_ins_ws',
       'wind_max_ins_ws_wd', 'wind.mix_wd', 'ta.avg_ta', 'ta.max_ta',
       'ta.min_ta', 'rn.sum_rn', 'rn.hr1_max_rn'],
      dtype='object')

In [ ]:
all_merged = all_merged.rename(columns={'cloud.ca' : 'ca', 'cloud.ch' : 'ch', 'wind.avg_ws' : 'avg_ws', 'wind.mix_ws' : 'mix_ws',
       'wind.max_ws' : 'max_ws', 'wind.max_wd' : 'max_wd', 'wind.max_ws_wd' : 'max_ws_wd', 'wind.max_ins_ws' : 'max_ins_ws',
       'wind_max_ins_ws_wd' : 'max_ins_ws_wd', 'wind.mix_wd' : 'mix_wd', 'ta.avg_ta' : 'avg_ta', 'ta.max_ta' : 'max_ta',
       'ta.min_ta' : 'min_ta', 'rn.sum_rn' : 'sum_rn', 'rn.hr1_max_rn' : 'hr1_max_rn'})

In [ ]:
all_merged.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/merged.csv')

# 태풍, 호우 유무 날짜 변수 추가해주기

In [ ]:
rain = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/rain_ver4.csv', index_col = 0)
typhoon = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/typoon_kr.csv', index_col = 1)
all_merged = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/merged.csv', index_col = 0)

In [ ]:
dates = list(typhoon['date'])

In [ ]:
for i in range(len(dates)):
  dates[i] = str(dates[i])

In [ ]:
for i in range(len(dates)): 
  if len(dates[i]) == 5 :
    dates[i] = '00' + dates[i]

In [ ]:
for i in range(len(dates)):
  if dates[i][0:2] == '00' :
    dates[i] = '20'+ dates[i]
  elif dates[i][0] == '1':
    dates[i] = '20'+ dates[i]
  elif dates[i][0] == '2':
    dates[i] = '20'+ dates[i]

In [ ]:
year = list(rain['year'])
month = list(rain['month'])
day = list(rain['day'])

for i in range(len(year)): 
  year[i] = str(year[i])
  month[i] = str(month[i])
  day[i] = str(day[i])

In [ ]:
ymd = []

for i in range(len(year)): 
  ymd.append(year[i] + month[i] + day[i])

In [ ]:
rain['yyyymmdd'] = ymd

In [ ]:
rain['yyyymmdd'] = pd.to_datetime(rain['yyyymmdd'], format='%Y%m%d')

In [ ]:
del rain['day']
del rain['month']
del rain['year']
rain = rain.rename(columns={'provin' : 'add', 'yyyymmdd' : 'date'})

In [ ]:
all_merged['date'] = pd.to_datetime(all_merged['date'], format='%Y-%m-%d')

In [ ]:
rain_yn = pd.DataFrame({'date' : dates_check, 'rain' : rain_yn})

In [ ]:
rain['rain_yn'] = 1

In [ ]:
rain_typhoon = rain.loc[:, rain.columns != '2021_damage']

In [ ]:
rain_typhoon

,wet_area,total_rain,duration_day,typoon,duration_hour,max_rain,day_rain,cum_rain_2,cum_rain_3,max_wind,avg_temp,add,district,typoon_dum,date,rain_yn
1,1.4,240.0,3,루사,6.0,4.0,14.0,14.0,14.0,2.1,26.700000,대구,남구,1,2002-08-30,1
2,1.4,240.0,3,루사,24.0,35.5,223.0,237.0,237.0,5.4,24.000000,대구,남구,1,2002-08-31,1
3,1.4,240.0,3,루사,24.0,2.0,3.0,226.0,240.0,5.2,23.900000,대구,남구,1,2002-09-01,1
4,31.8,148.5,2,매미,24.0,24.0,147.0,150.0,150.0,12.7,22.800000,대구,남구,1,2003-09-12,1
5,31.8,148.5,2,매미,24.0,0.5,1.5,148.5,151.5,8.5,25.300000,대구,남구,1,2003-09-13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23683,0.9,320.3,19,없음,0.0,0.0,0.0,0.0,8.8,3.1,22.812903,경북,봉화군,0,1998-08-14,1
23684,2.3,411.8,19,없음,0.0,0.0,0.0,6.5,15.5,6.6,25.102857,경북,영천시,0,1998-08-14,1
23685,965.2,243.8,3,루사,0.0,0.0,0.0,211.6,243.8,6.5,17.185714,강원,양양군,1,2002-09-01,1
23686,645.4,86.0,3,루사,0.0,0.0,0.0,85.7,86.0,5.8,21.633333,강원,정선군,1,2002-09-01,1


In [ ]:
rain_typhoon.columns

Index(['wet_area', 'total_rain', 'duration_day', 'typoon', 'duration_hour',
       'max_rain', 'day_rain', 'cum_rain_2', 'cum_rain_3', 'max_wind',
       'avg_temp', 'add', 'district', 'typoon_dum', 'date', 'rain_yn'],
      dtype='object')

In [ ]:
rt_final = rain_typhoon[['add', 'typoon_dum', 'date', 'rain_yn']]

In [ ]:
rt_final.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/호우요약.csv')

In [ ]:
rt_final

,add,typoon_dum,date,rain_yn
1,대구,1,2002-08-30,1
2,대구,1,2002-08-31,1
3,대구,1,2002-09-01,1
4,대구,1,2003-09-12,1
5,대구,1,2003-09-13,1
...,...,...,...,...
23683,경북,0,1998-08-14,1
23684,경북,0,1998-08-14,1
23685,강원,1,2002-09-01,1
23686,강원,1,2002-09-01,1


In [ ]:
dates_tobe = list(set(all_merged['date']))
dates_check = list(rain['date'])

In [ ]:
for i in range(len(dates_tobe)):
  dates_tobe[i] = str(dates_tobe[i])[0:10]

for i in range(len(dates_check)):
   dates_check[i] = str(dates_check[i])[0:10]

In [ ]:
#rain_yn_l = pd.DataFrame({'rain_yn_l' : rain_yn_l})
#our_dates = pd.DataFrame({'our_dates' : our_dates})

In [ ]:
yes_no_index = []
for i in range(len(dates_check)):
  if dates_check[i] in dates_tobe :
    yes_no_index.append(1)
  else:
    yes_no_index.append(0)

In [ ]:
rain['yes_no_index'] = yes_no_index

In [ ]:
rain_merging = rain[rain['yes_no_index'] == 1]
del rain_merging['yes_no_index']
rain_merging = rain_merging[['typoon', 'add', '2021_damage', 'date', 'rain_yn']]

In [ ]:
rain_merging = rain_merging.rename(columns={'typoon' : 'typhoon_yn', '2021_damage' : 'damage_conv'})
rain_merging['typhoon_yn'][rain_merging['typhoon_yn'] == '없음'] = 0
rain_merging['typhoon_yn'][rain_merging['typhoon_yn'] != 0] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
rain_merging

,typhoon_yn,add,damage_conv,date,rain_yn
6,0,대구,6.028112e+03,2016-07-01,1
7,0,대구,6.028112e+03,2016-07-02,1
8,0,대구,6.028112e+03,2016-07-03,1
9,0,대구,6.028112e+03,2016-07-04,1
10,0,대구,6.028112e+03,2016-07-05,1
...,...,...,...,...,...
23643,0,제주,1.393392e+07,2016-10-03,1
23644,0,제주,1.393392e+07,2016-10-04,1
23645,0,제주,1.393392e+07,2016-10-05,1
23646,0,제주,1.393392e+07,2016-10-06,1


In [ ]:
rain_merging.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/호우요약최종.csv')

In [ ]:
merged_fixing = pd.merge(all_merged, rain_merging, on = ['date', 'add'], how='outer')

In [ ]:
merged_fixing['typhoon_yn'][merged_fixing['typhoon_yn'].isna() == True] = 0
merged_fixing['damage_conv'][merged_fixing['damage_conv'].isna() == True] = 0
merged_fixing['rain_yn'][merged_fixing['rain_yn'].isna() == True] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
merged_fixing.isna().sum()

date             0
add              0
ca               0
ch               0
avg_ws           0
mix_ws           0
max_ws           0
max_wd           0
max_ws_wd        0
max_ins_ws       0
max_ins_ws_wd    0
mix_wd           0
avg_ta           0
max_ta           0
min_ta           0
sum_rn           0
hr1_max_rn       0
typhoon_yn       0
damage_conv      0
rain_yn          0
dtype: int64

In [ ]:
merged_final = merged_fixing

In [ ]:
merged_final.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/기상호우태풍병합데이터.csv')

In [ ]:
print(type(merged_final['typhoon_yn'][0]))  # typhoon_yn	rain_y
print(type(merged_final['rain_yn'][0]))

<class 'int'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [ ]:
merged_final['rain_yn'] = merged_final['rain_yn'].astype(int)

In [ ]:
merged_final

,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,max_ins_ws,max_ins_ws_wd,mix_wd,avg_ta,max_ta,min_ta,sum_rn,hr1_max_rn,typhoon_yn,damage_conv,rain_yn
0,2012-02-01,강원,6.650000,12.700000,3.005682,2.570455,6.090909,224.232955,240.344318,10.193182,245.621591,235.165909,-12.197727,-6.029545,-17.671591,0.040230,0.155172,0,0.0,0
1,2012-02-01,경기,3.500000,10.000000,2.246078,2.009259,4.882857,240.208333,253.354808,8.411881,250.224000,256.399020,-12.544954,-6.768519,-16.915741,0.000000,0.009174,0,0.0,0
2,2012-02-01,경남,6.296296,13.481481,4.020755,3.498113,7.264151,257.264151,234.711321,12.314815,238.038889,250.943396,-2.059259,4.559259,-7.512963,0.047170,0.179245,0,0.0,0
3,2012-02-01,경북,6.487179,12.863248,3.587324,3.238028,6.583099,278.873239,267.936620,11.548611,280.430556,280.119718,-6.623611,0.375000,-11.386111,0.592958,0.828169,0,0.0,0
4,2012-02-01,광주,5.857143,12.214286,3.216667,3.216667,6.650000,273.750000,264.616667,11.116667,270.666667,273.350000,-5.083333,1.000000,-7.683333,0.083333,0.666667,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56253,2022-10-24,세종,2.766667,23.102778,1.425000,1.100000,4.450000,286.875000,148.400000,9.300000,300.550000,310.200000,8.625000,15.575000,3.275000,0.000000,0.000000,0,0.0,0
56254,2022-10-25,세종,2.000000,16.000000,0.825000,0.575000,3.175000,112.500000,72.400000,6.025000,58.100000,46.550000,8.025000,17.200000,0.500000,0.000000,0.000000,0,0.0,0
56255,2022-10-26,세종,6.396296,59.212963,0.625000,0.300000,2.300000,118.125000,105.525000,3.950000,145.250000,132.850000,9.800000,18.700000,1.950000,0.000000,0.000000,0,0.0,0
56256,2022-10-27,세종,5.825397,55.130952,0.775000,0.500000,3.200000,185.625000,237.150000,7.400000,224.475000,182.950000,11.275000,20.150000,4.000000,0.000000,0.000000,0,0.0,0


이제 평범한 날, 호우는 있지만 태풍은 없는 날, 호우와 태풍 둘 다 있는 날을 구분해야 함

In [ ]:
df_with_notyphoon = merged_final[merged_final['typhoon_yn']==0]
df_with_rain = merged_final[merged_final['rain_yn']==1]

In [ ]:
#del df_with_notyphoon['typhoon_yn']
df_notyphoon_norain = df_with_notyphoon[df_with_notyphoon['rain_yn']==0]
del df_notyphoon_norain['rain_yn']
df_notyphoon_norain # 호우, 태풍 둘 다 없는 날

,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,max_ins_ws,max_ins_ws_wd,mix_wd,avg_ta,max_ta,min_ta,sum_rn,hr1_max_rn,damage_conv
0,2012-02-01,강원,6.650000,12.700000,3.005682,2.570455,6.090909,224.232955,240.344318,10.193182,245.621591,235.165909,-12.197727,-6.029545,-17.671591,0.040230,0.155172,0.0
1,2012-02-01,경기,3.500000,10.000000,2.246078,2.009259,4.882857,240.208333,253.354808,8.411881,250.224000,256.399020,-12.544954,-6.768519,-16.915741,0.000000,0.009174,0.0
2,2012-02-01,경남,6.296296,13.481481,4.020755,3.498113,7.264151,257.264151,234.711321,12.314815,238.038889,250.943396,-2.059259,4.559259,-7.512963,0.047170,0.179245,0.0
3,2012-02-01,경북,6.487179,12.863248,3.587324,3.238028,6.583099,278.873239,267.936620,11.548611,280.430556,280.119718,-6.623611,0.375000,-11.386111,0.592958,0.828169,0.0
4,2012-02-01,광주,5.857143,12.214286,3.216667,3.216667,6.650000,273.750000,264.616667,11.116667,270.666667,273.350000,-5.083333,1.000000,-7.683333,0.083333,0.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56253,2022-10-24,세종,2.766667,23.102778,1.425000,1.100000,4.450000,286.875000,148.400000,9.300000,300.550000,310.200000,8.625000,15.575000,3.275000,0.000000,0.000000,0.0
56254,2022-10-25,세종,2.000000,16.000000,0.825000,0.575000,3.175000,112.500000,72.400000,6.025000,58.100000,46.550000,8.025000,17.200000,0.500000,0.000000,0.000000,0.0
56255,2022-10-26,세종,6.396296,59.212963,0.625000,0.300000,2.300000,118.125000,105.525000,3.950000,145.250000,132.850000,9.800000,18.700000,1.950000,0.000000,0.000000,0.0
56256,2022-10-27,세종,5.825397,55.130952,0.775000,0.500000,3.200000,185.625000,237.150000,7.400000,224.475000,182.950000,11.275000,20.150000,4.000000,0.000000,0.000000,0.0


In [ ]:
df_notyphoon_yesrain = df_with_notyphoon[df_with_notyphoon['rain_yn']==1]
del df_notyphoon_yesrain['rain_yn']
df_notyphoon_yesrain # 호우는 내렸지만 태풍은 없던 날

,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,max_ins_ws,max_ins_ws_wd,mix_wd,avg_ta,max_ta,min_ta,sum_rn,hr1_max_rn,damage_conv
2300,2012-06-30,경기,8.534247,16.342466,2.494340,2.072222,5.686916,195.000000,192.633645,11.014151,193.809434,196.381132,22.130909,24.404587,20.335780,83.512037,16.930275,6.178129e+02
2301,2012-06-30,경기,8.534247,16.342466,2.494340,2.072222,5.686916,195.000000,192.633645,11.014151,193.809434,196.381132,22.130909,24.404587,20.335780,83.512037,16.930275,1.235626e+03
2302,2012-06-30,경기,8.534247,16.342466,2.494340,2.072222,5.686916,195.000000,192.633645,11.014151,193.809434,196.381132,22.130909,24.404587,20.335780,83.512037,16.930275,3.089064e+03
2381,2012-07-05,강원,7.804878,18.695122,1.463636,0.923864,4.127273,194.573864,195.073864,6.597727,199.526136,202.963636,22.335632,26.137500,19.898864,5.143182,2.495455,1.557403e+04
2382,2012-07-05,경기,8.659091,16.454545,1.058333,0.575229,3.885185,153.577982,181.596296,7.040187,178.364486,171.443519,22.938739,25.678378,21.247748,33.789908,15.735455,1.091809e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24598,2016-10-06,전북,3.857143,5.857143,2.107143,1.607143,4.711905,239.464286,202.011905,7.000000,240.621429,228.971429,17.825000,23.383333,13.447619,0.000000,0.000000,0.000000e+00
24599,2016-10-06,제주,4.944444,25.277778,2.635000,1.842500,5.135000,203.625000,159.467500,8.455000,162.470000,155.372500,20.107317,24.127500,16.700000,0.025000,0.025000,1.393392e+07
53208,2012-08-14,세종,6.140981,14.969710,2.125000,1.775000,4.775000,118.125000,124.025000,7.475000,130.100000,122.150000,25.900000,30.575000,21.175000,2.250000,2.250000,5.332240e+04
53209,2012-08-15,세종,8.009266,17.300276,2.350000,1.725000,6.225000,202.500000,232.300000,11.725000,204.625000,195.375000,25.625000,29.900000,22.775000,138.500000,60.500000,5.332240e+04


In [ ]:
df_yesrain_yestyphoon = df_with_rain[df_with_rain['typhoon_yn'] == 1]
del df_yesrain_yestyphoon['typhoon_yn']
del df_yesrain_yestyphoon['rain_yn']
df_yesrain_yestyphoon   # 호우랑 태풍 둘다 있었던 날. 정확하게는 태풍이 있어서 호우가 있었던 날

,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,max_ins_ws,max_ins_ws_wd,mix_wd,avg_ta,max_ta,min_ta,sum_rn,hr1_max_rn,damage_conv
2674,2012-07-17,경기,7.842105,20.342105,0.943925,0.601835,3.158333,106.513761,116.263889,5.438679,129.533019,104.556075,24.348182,29.230000,19.899091,0.124545,0.124545,7.413754e+03
2676,2012-07-17,경기,7.842105,20.342105,0.943925,0.601835,3.158333,106.513761,116.263889,5.438679,129.533019,104.556075,24.348182,29.230000,19.899091,0.124545,0.124545,1.853439e+03
2683,2012-07-17,서울,8.085714,19.028571,1.420690,0.910345,3.379310,92.327586,110.379310,5.813793,105.379310,96.882759,25.210345,29.234483,21.082759,0.034483,0.034483,6.178129e+02
2684,2012-07-17,서울,8.085714,19.028571,1.420690,0.910345,3.379310,92.327586,110.379310,5.813793,105.379310,96.882759,25.210345,29.234483,21.082759,0.034483,0.034483,1.235626e+03
2686,2012-07-17,인천,6.692308,22.717949,1.588000,1.144000,3.572000,105.300000,111.308000,5.508000,115.792000,110.420000,23.761538,27.660000,20.244000,0.060000,0.060000,1.853439e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18417,2015-08-27,부산,4.156250,27.718750,2.657143,2.385714,5.814286,233.035714,213.000000,8.185714,222.742857,232.542857,24.093333,28.278571,20.571429,0.000000,0.000000,1.344945e+05
18418,2015-08-27,부산,4.156250,27.718750,2.657143,2.385714,5.814286,233.035714,213.000000,8.185714,222.742857,232.542857,24.093333,28.278571,20.571429,0.000000,0.000000,3.288000e+05
18420,2015-08-27,울산,3.375000,22.500000,2.744444,2.400000,5.622222,240.000000,228.822222,8.888889,232.722222,236.822222,23.920000,29.177778,18.811111,0.000000,0.000000,1.344945e+05
18421,2015-08-27,울산,3.375000,22.500000,2.744444,2.400000,5.622222,240.000000,228.822222,8.888889,232.722222,236.822222,23.920000,29.177778,18.811111,0.000000,0.000000,3.288000e+05


In [ ]:
df_notyphoon_norain.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/df_notyphoon_norain.csv')
df_notyphoon_yesrain.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/df_notyphoon_yesrain.csv')
df_yesrain_yestyphoon.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/df_yesrain_yestyphoon.csv')

In [ ]:
merged_final

,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,max_ins_ws,max_ins_ws_wd,mix_wd,avg_ta,max_ta,min_ta,sum_rn,hr1_max_rn,typhoon_yn,damage_conv,rain_yn
0,2012-02-01,강원,6.650000,12.700000,3.005682,2.570455,6.090909,224.232955,240.344318,10.193182,245.621591,235.165909,-12.197727,-6.029545,-17.671591,0.040230,0.155172,0,0.0,0
1,2012-02-01,경기,3.500000,10.000000,2.246078,2.009259,4.882857,240.208333,253.354808,8.411881,250.224000,256.399020,-12.544954,-6.768519,-16.915741,0.000000,0.009174,0,0.0,0
2,2012-02-01,경남,6.296296,13.481481,4.020755,3.498113,7.264151,257.264151,234.711321,12.314815,238.038889,250.943396,-2.059259,4.559259,-7.512963,0.047170,0.179245,0,0.0,0
3,2012-02-01,경북,6.487179,12.863248,3.587324,3.238028,6.583099,278.873239,267.936620,11.548611,280.430556,280.119718,-6.623611,0.375000,-11.386111,0.592958,0.828169,0,0.0,0
4,2012-02-01,광주,5.857143,12.214286,3.216667,3.216667,6.650000,273.750000,264.616667,11.116667,270.666667,273.350000,-5.083333,1.000000,-7.683333,0.083333,0.666667,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56253,2022-10-24,세종,2.766667,23.102778,1.425000,1.100000,4.450000,286.875000,148.400000,9.300000,300.550000,310.200000,8.625000,15.575000,3.275000,0.000000,0.000000,0,0.0,0
56254,2022-10-25,세종,2.000000,16.000000,0.825000,0.575000,3.175000,112.500000,72.400000,6.025000,58.100000,46.550000,8.025000,17.200000,0.500000,0.000000,0.000000,0,0.0,0
56255,2022-10-26,세종,6.396296,59.212963,0.625000,0.300000,2.300000,118.125000,105.525000,3.950000,145.250000,132.850000,9.800000,18.700000,1.950000,0.000000,0.000000,0,0.0,0
56256,2022-10-27,세종,5.825397,55.130952,0.775000,0.500000,3.200000,185.625000,237.150000,7.400000,224.475000,182.950000,11.275000,20.150000,4.000000,0.000000,0.000000,0,0.0,0


# Train&Test split

In [ ]:
merged_training = merged_final.iloc[:48857]
merged_test = merged_final.iloc[48857:]

In [ ]:
train_with_notyphoon = merged_training[merged_training['typhoon_yn']==0]
train_with_rain = merged_training[merged_training['rain_yn']==1]

In [ ]:
train_with_notyphoon = merged_training[merged_training['typhoon_yn']==0]
train_with_rain = merged_training[merged_training['rain_yn']==1]

train_notyphoon_norain = train_with_notyphoon[train_with_notyphoon['rain_yn']==0]
del train_notyphoon_norain['rain_yn']
del train_notyphoon_norain['typhoon_yn']
train_notyphoon_norain # 호우, 태풍 둘 다 없는 날

train_notyphoon_yesrain = train_with_notyphoon[train_with_notyphoon['rain_yn']==1]
del train_notyphoon_yesrain['rain_yn']
del train_notyphoon_yesrain['typhoon_yn']
train_notyphoon_yesrain # 호우는 내렸지만 태풍은 없던 날

train_yesrain_yestyphoon = train_with_rain[train_with_rain['typhoon_yn'] == 1]
del train_yesrain_yestyphoon['typhoon_yn']
del train_yesrain_yestyphoon['rain_yn']
train_yesrain_yestyphoon   # 호우랑 태풍 둘다 있었던 날. 정확하게는 태풍이 있어서 호우가 있었던 날

,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,max_ins_ws,max_ins_ws_wd,mix_wd,avg_ta,max_ta,min_ta,sum_rn,hr1_max_rn,damage_conv
2674,2012-07-17,경기,7.842105,20.342105,0.943925,0.601835,3.158333,106.513761,116.263889,5.438679,129.533019,104.556075,24.348182,29.230000,19.899091,0.124545,0.124545,7.413754e+03
2676,2012-07-17,경기,7.842105,20.342105,0.943925,0.601835,3.158333,106.513761,116.263889,5.438679,129.533019,104.556075,24.348182,29.230000,19.899091,0.124545,0.124545,1.853439e+03
2683,2012-07-17,서울,8.085714,19.028571,1.420690,0.910345,3.379310,92.327586,110.379310,5.813793,105.379310,96.882759,25.210345,29.234483,21.082759,0.034483,0.034483,6.178129e+02
2684,2012-07-17,서울,8.085714,19.028571,1.420690,0.910345,3.379310,92.327586,110.379310,5.813793,105.379310,96.882759,25.210345,29.234483,21.082759,0.034483,0.034483,1.235626e+03
2686,2012-07-17,인천,6.692308,22.717949,1.588000,1.144000,3.572000,105.300000,111.308000,5.508000,115.792000,110.420000,23.761538,27.660000,20.244000,0.060000,0.060000,1.853439e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18417,2015-08-27,부산,4.156250,27.718750,2.657143,2.385714,5.814286,233.035714,213.000000,8.185714,222.742857,232.542857,24.093333,28.278571,20.571429,0.000000,0.000000,1.344945e+05
18418,2015-08-27,부산,4.156250,27.718750,2.657143,2.385714,5.814286,233.035714,213.000000,8.185714,222.742857,232.542857,24.093333,28.278571,20.571429,0.000000,0.000000,3.288000e+05
18420,2015-08-27,울산,3.375000,22.500000,2.744444,2.400000,5.622222,240.000000,228.822222,8.888889,232.722222,236.822222,23.920000,29.177778,18.811111,0.000000,0.000000,1.344945e+05
18421,2015-08-27,울산,3.375000,22.500000,2.744444,2.400000,5.622222,240.000000,228.822222,8.888889,232.722222,236.822222,23.920000,29.177778,18.811111,0.000000,0.000000,3.288000e+05


In [ ]:
test_with_notyphoon = merged_test[merged_test['typhoon_yn']==0]
test_with_rain = merged_test[merged_test['rain_yn']==1]

test_notyphoon_norain = test_with_notyphoon[test_with_notyphoon['rain_yn']==0]
del test_notyphoon_norain['rain_yn']
del test_notyphoon_norain['typhoon_yn']
test_notyphoon_norain # 호우, 태풍 둘 다 없는 날

test_notyphoon_yesrain = test_with_notyphoon[test_with_notyphoon['rain_yn']==1]
del test_notyphoon_yesrain['rain_yn']
del test_notyphoon_yesrain['typhoon_yn']
test_notyphoon_yesrain # 호우는 내렸지만 태풍은 없던 날

test_yesrain_yestyphoon = test_with_rain[test_with_rain['typhoon_yn'] == 1]
del test_yesrain_yestyphoon['typhoon_yn']
del test_yesrain_yestyphoon['rain_yn']
test_yesrain_yestyphoon   # 호우랑 태풍 둘다 있었던 날. 정확하게는 태풍이 있어서 호우가 있었던 날

,date,add,ca,ch,avg_ws,mix_ws,max_ws,max_wd,max_ws_wd,max_ins_ws,max_ins_ws_wd,mix_wd,avg_ta,max_ta,min_ta,sum_rn,hr1_max_rn,damage_conv


# 2022년에 태풍이 없었다고 나오네.. 포항있어야 할텐데...?

In [ ]:
train_notyphoon_norain.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/train_notyphoon_norain.csv')
train_notyphoon_yesrain.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/train_notyphoon_yesrain.csv')
train_yesrain_yestyphoon.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/train_yesrain_yestyphoon.csv')

test_notyphoon_norain. to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/test_notyphoon_norain.csv')
test_notyphoon_yesrain.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/test_notyphoon_yesrain.csv')
test_yesrain_yestyphoon.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/test_yesrain_yestyphoon.csv')

In [ ]:
merged_training.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/merged_training.csv')
merged_test.to_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/merged_test.csv')